In [1]:
import os
import shutil
import numpy as np
import pandas as pd
import seaborn as sns 
import cosmicfish as cf 
import matplotlib.pyplot as plt
import dill


# Instruct pyplot to use seaborn 
sns.set()

# Set project, data, CLASS directories 
projectdir = cf.correct_path("/Volumes/SSD01/")
datastore = cf.correct_path("/Volumes/SSD01/data.nosync16c/")
classpath = os.path.join("/Users/nicholasdeporzio/Desktop/cfworkspace/class4")

# Specify resolution of numerical integrals
derivative_step = 0.008 # How much to vary parameter to calculate numerical derivative
mu_integral_step = 0.05 # For calculating numerical integral wrt mu between -1 and 1 

# Generate output paths  
ps16_resultsdir = os.path.join(projectdir, 'results', 'ps16c')
cf.makedirectory(ps16_resultsdir)

# Linda Fiducial Cosmology 
ps16_fid = {
        "A_s" : 2.2321e-9, 
        "n_s" : 0.967,
        "omega_b" : 0.02226,
        "omega_cdm" : 0.1127,
        "tau_reio" : 0.0598,
        "h" : 0.701,
        "T_cmb" : 2.726, # Units [K]
        "N_ncdm" : 1., 
        "deg_ncdm" : 2.0,
        "T_ncdm" : (0.9052/2.726), # Units [T_cmb]. 
        "m_ncdm" : 0.0328, # Units [eV]
        "b0" : 1.0, 
        "beta0" : 1.7, 
        "beta1" : 1.0,
        "alphak2" : 1.0,
        "sigma_fog_0" : 250000, #Units [m s^-2]
        "N_eff" : 3.046, #We allow relativistic neutrinos in addition to our DM relic
        "relic_vary" : "N_ncdm" # Fix T_ncdm or m_ncdm 
        }

# EUCLID Parameters
z_table = np.array([0.65, 0.75, 0.85, 0.95, 1.05, 1.15, 1.25, 1.35, 1.45, 1.55, 1.65, 1.75, 1.85, 1.95])
dNdz = np.array([2434.280, 4364.812, 4728.559, 4825.798, 4728.797, 4507.625, 4269.851, 3720.657, 3104.309, 
    2308.975, 1514.831, 1474.707, 893.716, 497.613])
skycover = 0.3636 # Sky coverage of survey in fraction

# Run Fisher Forecast
#masses = np.append(np.array([0.001, 0.01]), np.geomspace(0.1, 22.0, 25))
masses = np.array([0.2])

omegacdm_set = ps16_fid['omega_cdm'] - ps16_fid["N_ncdm"]*((masses/cf.NEUTRINO_SCALE_FACTOR)*np.power(ps16_fid['T_ncdm']*2.726 / 1.95, 3.))                                     
ps16_fiducialset = [dict(ps16_fid, **{'m_ncdm' : masses[midx], 'omega_cdm' : omegacdm_set[midx]}) 
               for midx, mval in enumerate(masses)]


In [2]:
ps16_forecastset = [cf.forecast(
    classpath, 
    datastore, 
    'relic', 
    fidval, 
    z_table, 
    "EUCLID",
    dNdz, 
    fsky=skycover, 
    dstep=derivative_step,
    RSD=True,
    FOG=True,
    AP=True,
    COV=True) for fididx, fidval in enumerate(ps16_fiducialset)]
for fidx, fcst in enumerate(ps16_forecastset): 
    fcst.gen_pm()
    fcst.gen_fisher(
        fisher_order=[
            'omega_b',                                    
            'omega_cdm',                                  
            'n_s',                                        
            'A_s',                                        
            'tau_reio',                                   
            'h',                                                                             
            'N_ncdm',                                 
            'sigma_fog',                                   
            'beta0',
            'beta1',
            'alpha_k2'],
        mu_step=mu_integral_step, 
        skipgen=False)
    print("Relic Forecast ", fidx, " complete...")
    dill.dump_session(os.path.join(ps16_resultsdir, 'ps16c.db'))

#A_s-->A_s = 2.2321e-09
#n_s-->n_s = 0.967
#omega_b-->omega_b = 0.02226
#omega_cdm-->omega_cdm = 0.11248520529863264
#tau_reio-->tau_reio = 0.0598
#h-->h = 0.701
#T_cmb-->T_cmb = 2.726
#N_ncdm-->N_ncdm = 1.0
#deg_ncdm-->deg_ncdm = 2.0
#T_ncdm-->T_ncdm = 0.33206162876008805
#m_ncdm-->m_ncdm = 0.2
#N_eff-->N_eff = 3.046
#z_pk-->z_pk = 0.65
Dataset generated at: /Volumes/SSD01/data.nosync16c/1586581928.94682120.077585
#A_s-->A_s = 2.2321e-09
#n_s-->n_s = 0.967
#omega_b-->omega_b = 0.02226
#omega_cdm-->omega_cdm = 0.11248520529863264
#tau_reio-->tau_reio = 0.0598
#h-->h = 0.701
#T_cmb-->T_cmb = 2.726
#N_ncdm-->N_ncdm = 1.0
#deg_ncdm-->deg_ncdm = 2.0
#T_ncdm-->T_ncdm = 0.33206162876008805
#m_ncdm-->m_ncdm = 0.2
#N_eff-->N_eff = 3.046
#z_pk-->z_pk = 0.75
Dataset generated at: /Volumes/SSD01/data.nosync16c/1586581929.3718440.868006
#A_s-->A_s = 2.2321e-09
#n_s-->n_s = 0.967
#omega_b-->omega_b = 0.02226
#omega_cdm-->omega_cdm = 0.11248520529863264
#tau_reio-->tau_reio = 0.0598
#h-->h = 0.701
#

Dataset generated at: /Volumes/SSD01/data.nosync16c/1586581937.5764250.815813
#A_s-->A_s = 2.2321e-09
#n_s-->n_s = 0.967
#omega_b-->omega_b = 0.02243808
#omega_cdm-->omega_cdm = 0.11248520529863264
#tau_reio-->tau_reio = 0.0598
#h-->h = 0.701
#T_cmb-->T_cmb = 2.726
#N_ncdm-->N_ncdm = 1.0
#deg_ncdm-->deg_ncdm = 2.0
#T_ncdm-->T_ncdm = 0.33206162876008805
#m_ncdm-->m_ncdm = 0.2
#N_eff-->N_eff = 3.046
#z_pk-->z_pk = 1.25
Dataset generated at: /Volumes/SSD01/data.nosync16c/1586581937.9931320.474323
#A_s-->A_s = 2.2321e-09
#n_s-->n_s = 0.967
#omega_b-->omega_b = 0.02243808
#omega_cdm-->omega_cdm = 0.11248520529863264
#tau_reio-->tau_reio = 0.0598
#h-->h = 0.701
#T_cmb-->T_cmb = 2.726
#N_ncdm-->N_ncdm = 1.0
#deg_ncdm-->deg_ncdm = 2.0
#T_ncdm-->T_ncdm = 0.33206162876008805
#m_ncdm-->m_ncdm = 0.2
#N_eff-->N_eff = 3.046
#z_pk-->z_pk = 1.35
Dataset generated at: /Volumes/SSD01/data.nosync16c/1586581938.3989560.619238
#A_s-->A_s = 2.2321e-09
#n_s-->n_s = 0.967
#omega_b-->omega_b = 0.02243808
#omeg

Dataset generated at: /Volumes/SSD01/data.nosync16c/1586581946.63279410.382615
#A_s-->A_s = 2.2321e-09
#n_s-->n_s = 0.967
#omega_b-->omega_b = 0.022081919999999998
#omega_cdm-->omega_cdm = 0.11248520529863264
#tau_reio-->tau_reio = 0.0598
#h-->h = 0.701
#T_cmb-->T_cmb = 2.726
#N_ncdm-->N_ncdm = 1.0
#deg_ncdm-->deg_ncdm = 2.0
#T_ncdm-->T_ncdm = 0.33206162876008805
#m_ncdm-->m_ncdm = 0.2
#N_eff-->N_eff = 3.046
#z_pk-->z_pk = 1.85
Dataset generated at: /Volumes/SSD01/data.nosync16c/1586581947.0980720.874514
#A_s-->A_s = 2.2321e-09
#n_s-->n_s = 0.967
#omega_b-->omega_b = 0.022081919999999998
#omega_cdm-->omega_cdm = 0.11248520529863264
#tau_reio-->tau_reio = 0.0598
#h-->h = 0.701
#T_cmb-->T_cmb = 2.726
#N_ncdm-->N_ncdm = 1.0
#deg_ncdm-->deg_ncdm = 2.0
#T_ncdm-->T_ncdm = 0.33206162876008805
#m_ncdm-->m_ncdm = 0.2
#N_eff-->N_eff = 3.046
#z_pk-->z_pk = 1.95
Dataset generated at: /Volumes/SSD01/data.nosync16c/1586581947.5422210.058111
#A_s-->A_s = 2.2321e-09
#n_s-->n_s = 0.967
#omega_b-->omega

Dataset generated at: /Volumes/SSD01/data.nosync16c/1586581957.8174890.222977
#A_s-->A_s = 2.2321e-09
#n_s-->n_s = 0.967
#omega_b-->omega_b = 0.02226
#omega_cdm-->omega_cdm = 0.11158532365624357
#tau_reio-->tau_reio = 0.0598
#h-->h = 0.701
#T_cmb-->T_cmb = 2.726
#N_ncdm-->N_ncdm = 1.0
#deg_ncdm-->deg_ncdm = 2.0
#T_ncdm-->T_ncdm = 0.33206162876008805
#m_ncdm-->m_ncdm = 0.2
#N_eff-->N_eff = 3.046
#z_pk-->z_pk = 1.05
Dataset generated at: /Volumes/SSD01/data.nosync16c/1586581958.5906830.807565
#A_s-->A_s = 2.2321e-09
#n_s-->n_s = 0.967
#omega_b-->omega_b = 0.02226
#omega_cdm-->omega_cdm = 0.11158532365624357
#tau_reio-->tau_reio = 0.0598
#h-->h = 0.701
#T_cmb-->T_cmb = 2.726
#N_ncdm-->N_ncdm = 1.0
#deg_ncdm-->deg_ncdm = 2.0
#T_ncdm-->T_ncdm = 0.33206162876008805
#m_ncdm-->m_ncdm = 0.2
#N_eff-->N_eff = 3.046
#z_pk-->z_pk = 1.15
Dataset generated at: /Volumes/SSD01/data.nosync16c/1586581959.68692780.033681
#A_s-->A_s = 2.2321e-09
#n_s-->n_s = 0.967
#omega_b-->omega_b = 0.02226
#omega_cdm-->

Dataset generated at: /Volumes/SSD01/data.nosync16c/1586581975.9582380.132411
#A_s-->A_s = 2.2321e-09
#n_s-->n_s = 0.967
#omega_b-->omega_b = 0.02226
#omega_cdm-->omega_cdm = 0.11248520529863264
#tau_reio-->tau_reio = 0.0598
#h-->h = 0.706608
#T_cmb-->T_cmb = 2.726
#N_ncdm-->N_ncdm = 1.0
#deg_ncdm-->deg_ncdm = 2.0
#T_ncdm-->T_ncdm = 0.33206162876008805
#m_ncdm-->m_ncdm = 0.2
#N_eff-->N_eff = 3.046
#z_pk-->z_pk = 1.65
Dataset generated at: /Volumes/SSD01/data.nosync16c/1586581977.030920.897159
#A_s-->A_s = 2.2321e-09
#n_s-->n_s = 0.967
#omega_b-->omega_b = 0.02226
#omega_cdm-->omega_cdm = 0.11248520529863264
#tau_reio-->tau_reio = 0.0598
#h-->h = 0.706608
#T_cmb-->T_cmb = 2.726
#N_ncdm-->N_ncdm = 1.0
#deg_ncdm-->deg_ncdm = 2.0
#T_ncdm-->T_ncdm = 0.33206162876008805
#m_ncdm-->m_ncdm = 0.2
#N_eff-->N_eff = 3.046
#z_pk-->z_pk = 1.75
Dataset generated at: /Volumes/SSD01/data.nosync16c/1586581978.16892390.129590
#A_s-->A_s = 2.2321e-09
#n_s-->n_s = 0.967
#omega_b-->omega_b = 0.02226
#omega_c

#A_s-->A_s = 2.2321e-09
#n_s-->n_s = 0.967
#omega_b-->omega_b = 0.02226
#omega_cdm-->omega_cdm = 0.11248520529863264
#tau_reio-->tau_reio = 0.0602784
#h-->h = 0.701
#T_cmb-->T_cmb = 2.726
#N_ncdm-->N_ncdm = 1.0
#deg_ncdm-->deg_ncdm = 2.0
#T_ncdm-->T_ncdm = 0.33206162876008805
#m_ncdm-->m_ncdm = 0.2
#N_eff-->N_eff = 3.046
#z_pk-->z_pk = 0.75
Dataset generated at: /Volumes/SSD01/data.nosync16c/1586581999.3217040.734428
#A_s-->A_s = 2.2321e-09
#n_s-->n_s = 0.967
#omega_b-->omega_b = 0.02226
#omega_cdm-->omega_cdm = 0.11248520529863264
#tau_reio-->tau_reio = 0.0602784
#h-->h = 0.701
#T_cmb-->T_cmb = 2.726
#N_ncdm-->N_ncdm = 1.0
#deg_ncdm-->deg_ncdm = 2.0
#T_ncdm-->T_ncdm = 0.33206162876008805
#m_ncdm-->m_ncdm = 0.2
#N_eff-->N_eff = 3.046
#z_pk-->z_pk = 0.85
Dataset generated at: /Volumes/SSD01/data.nosync16c/1586582000.24316380.510211
#A_s-->A_s = 2.2321e-09
#n_s-->n_s = 0.967
#omega_b-->omega_b = 0.02226
#omega_cdm-->omega_cdm = 0.11248520529863264
#tau_reio-->tau_reio = 0.0602784
#h-->h 

Dataset generated at: /Volumes/SSD01/data.nosync16c/1586582026.3657810.308808
#A_s-->A_s = 2.2321e-09
#n_s-->n_s = 0.967
#omega_b-->omega_b = 0.02226
#omega_cdm-->omega_cdm = 0.11248520529863264
#tau_reio-->tau_reio = 0.059321599999999995
#h-->h = 0.701
#T_cmb-->T_cmb = 2.726
#N_ncdm-->N_ncdm = 1.0
#deg_ncdm-->deg_ncdm = 2.0
#T_ncdm-->T_ncdm = 0.33206162876008805
#m_ncdm-->m_ncdm = 0.2
#N_eff-->N_eff = 3.046
#z_pk-->z_pk = 1.35
Dataset generated at: /Volumes/SSD01/data.nosync16c/1586582027.7618930.649990
#A_s-->A_s = 2.2321e-09
#n_s-->n_s = 0.967
#omega_b-->omega_b = 0.02226
#omega_cdm-->omega_cdm = 0.11248520529863264
#tau_reio-->tau_reio = 0.059321599999999995
#h-->h = 0.701
#T_cmb-->T_cmb = 2.726
#N_ncdm-->N_ncdm = 1.0
#deg_ncdm-->deg_ncdm = 2.0
#T_ncdm-->T_ncdm = 0.33206162876008805
#m_ncdm-->m_ncdm = 0.2
#N_eff-->N_eff = 3.046
#z_pk-->z_pk = 1.45
Dataset generated at: /Volumes/SSD01/data.nosync16c/1586582028.6356520.540515
#A_s-->A_s = 2.2321e-09
#n_s-->n_s = 0.967
#omega_b-->omeg

Dataset generated at: /Volumes/SSD01/data.nosync16c/1586582055.8066220.628757
#A_s-->A_s = 2.2321e-09
#n_s-->n_s = 0.967
#omega_b-->omega_b = 0.02226
#omega_cdm-->omega_cdm = 0.11248520529863264
#tau_reio-->tau_reio = 0.0598
#h-->h = 0.701
#T_cmb-->T_cmb = 2.726
#N_ncdm-->N_ncdm = 1.0
#deg_ncdm-->deg_ncdm = 2.016
#T_ncdm-->T_ncdm = 0.33206162876008805
#m_ncdm-->m_ncdm = 0.2
#N_eff-->N_eff = 3.046
#z_pk-->z_pk = 1.95
Dataset generated at: /Volumes/SSD01/data.nosync16c/1586582057.4394560.735842
#A_s-->A_s = 2.2321e-09
#n_s-->n_s = 0.967
#omega_b-->omega_b = 0.02226
#omega_cdm-->omega_cdm = 0.11248520529863264
#tau_reio-->tau_reio = 0.0598
#h-->h = 0.701
#T_cmb-->T_cmb = 2.726
#N_ncdm-->N_ncdm = 1.0
#deg_ncdm-->deg_ncdm = 1.984
#T_ncdm-->T_ncdm = 0.33206162876008805
#m_ncdm-->m_ncdm = 0.2
#N_eff-->N_eff = 3.046
#z_pk-->z_pk = 0.65
Dataset generated at: /Volumes/SSD01/data.nosync16c/1586582058.8173860.486064
#A_s-->A_s = 2.2321e-09
#n_s-->n_s = 0.967
#omega_b-->omega_b = 0.02226
#omega_cdm

Fisher element ( 4 ,  4 ), ( 4 ,  4 ) calculated...
Fisher element ( 4 ,  5 ), ( 5 ,  4 ) calculated...
Fisher element ( 4 ,  6 ), ( 6 ,  4 ) calculated...
Fisher element ( 4 ,  7 ), ( 7 ,  4 ) calculated...
Fisher element ( 4 ,  8 ), ( 8 ,  4 ) calculated...
Fisher element ( 4 ,  9 ), ( 9 ,  4 ) calculated...
Fisher element ( 4 ,  10 ), ( 10 ,  4 ) calculated...
Fisher element ( 5 ,  5 ), ( 5 ,  5 ) calculated...
Fisher element ( 5 ,  6 ), ( 6 ,  5 ) calculated...
Fisher element ( 5 ,  7 ), ( 7 ,  5 ) calculated...
Fisher element ( 5 ,  8 ), ( 8 ,  5 ) calculated...
Fisher element ( 5 ,  9 ), ( 9 ,  5 ) calculated...
Fisher element ( 5 ,  10 ), ( 10 ,  5 ) calculated...
Fisher element ( 6 ,  6 ), ( 6 ,  6 ) calculated...
Fisher element ( 6 ,  7 ), ( 7 ,  6 ) calculated...
Fisher element ( 6 ,  8 ), ( 8 ,  6 ) calculated...
Fisher element ( 6 ,  9 ), ( 9 ,  6 ) calculated...
Fisher element ( 6 ,  10 ), ( 10 ,  6 ) calculated...
Fisher element ( 7 ,  7 ), ( 7 ,  7 ) calculated...
Fisher

In [ ]:
dill.load_session(os.path.join(ps16_resultsdir, 'ps16c.db'))

In [ ]:
ps16_forecastset[0].spectra_mid[0].datadirectory

In [3]:
np.savetxt("/Users/nicholasdeporzio/Desktop/k_3.txt", ps16_forecastset[0].k_table[0], delimiter='\t')
np.savetxt("/Users/nicholasdeporzio/Desktop/db_3.txt", ps16_forecastset[0].spectra_mid[0].b_interp_table, delimiter='\t')
np.savetxt("/Users/nicholasdeporzio/Desktop/dcdm_3.txt", ps16_forecastset[0].spectra_mid[0].cdm_interp_table, delimiter='\t')
np.savetxt("/Users/nicholasdeporzio/Desktop/Pg_3.txt", ps16_forecastset[0].Pg[0, :, 20], delimiter='\t')
np.savetxt("/Users/nicholasdeporzio/Desktop/Pm_3.txt", ps16_forecastset[0].Pm[0], delimiter='\t')

In [ ]:
plt.figure(figsize=(15,7.5))

plt.loglog(ps16_forecastset[0].k_table[0], ps16_forecastset[0].Pg[0, :, 20], label=r"$P_g$, $g_\chi$=2.0", linestyle='solid')






plt.title(r"DESI, z=0.65, $\mu=0$, m=0.2[eV]")
plt.xlabel(r"$k$ [Mpc$^{-1}$]", fontsize=24)
plt.ylabel(r"$P_g$", fontsize=24)
plt.legend(fontsize=18, loc='lower left')
plt.tick_params(axis='x', which='minttor')
plt.grid(True, which='minor')
plt.savefig("/Users/nicholasdeporzio/Desktop/testing/plotA19c.png")
plt.show()